### Importing the Required Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
print(tf.__version__)

2.4.1


In [ ]:
from keras.layers import LSTM
from keras.optimizers import SGD
from keras.layers import GRU
from tensorflow.keras.utils import to_categorical

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
import os
import csv

### Importing and Reading the Dataset

In [ ]:
!git clone https://github.com/harnoor-puniyani/Movie-genre-classification-dataset

Cloning into 'Movie-genre-classification-dataset'...
remote: Enumerating objects: 7873, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7873 (delta 0), reused 1 (delta 0), pack-reused 7872
Receiving objects: 100% (7873/7873), 246.59 MiB | 31.15 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (7871/7871), done.


In [ ]:
data = pd.read_csv('/content/Movies-Poster_Dataset/train.csv')

###Splitting CSV file

In [ ]:
def split_csv(filehandler, delimiter=',', row_limit=900, 
    output_name_template='output_%s.csv', output_path='.', keep_headers=True):
  reader = csv.reader(filehandler, delimiter=delimiter)
  current_piece = 1
  current_out_path = os.path.join(
    output_path,
    output_name_template  % current_piece
  )
  current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
  current_limit = row_limit
  if keep_headers:
    headers = next(reader)
    current_out_writer.writerow(headers)
  for i, row in enumerate(reader):
    if i + 1 > current_limit:
        current_piece += 1
        current_limit = row_limit * current_piece
        current_out_path = os.path.join(
           output_path,
           output_name_template  % current_piece
        )
        current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
        if keep_headers:
            current_out_writer.writerow(headers)
    current_out_writer.writerow(row)

In [ ]:
split_csv(open('/content/Movies-Poster_Dataset/train.csv','r'));

In [ ]:
data1 = pd.read_csv('/content/output_1.csv')

In [ ]:
data1.shape

(900, 27)

###Pre-processing Data

In [ ]:
img_width = 350
img_height = 350

x =[]

  
for i in tqdm(range(data1.shape[0])):
  path = '/content/Movies-Poster_Dataset/Images/' + data1['Id'][i] + '.jpg'
  img = image.load_img(path, target_size=(img_width,img_height,3))
  img = image.img_to_array(img)
  img = img/255.0
  x.append(img)
x=np.array(x)
# image.save_img(path = '/content/processed_data/', x, data_format=None, file_format='jpg', scale=False)


100%|██████████| 900/900 [00:08<00:00, 106.28it/s]


### Splitting Data for Training and Testing

In [ ]:
y = data1.drop(['Id', 'Genre'], axis = 1)
y = y.to_numpy()
y.shape

(900, 25)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.25)

In [ ]:
x_train[0].shape

(350, 350, 3)

### Reshaping Data for RNN

In [ ]:
nsamples, nx, ny ,nz= x_train.shape
xx_train = x_train.reshape((nsamples,1,nx*ny*nz))
xx_train.shape

(675, 1, 367500)

In [ ]:
nsamples, nx, ny ,nz= x_test.shape
xx_test = x_test.reshape((nsamples,1,nx*ny*nz))

In [ ]:
y_train.shape

(675, 25)

In [ ]:
yy_train = to_categorical(y_train, 25)
yy_test = to_categorical(y_test, 25)


In [ ]:
yy_train.shape

(675, 25, 25)

###Defining and Training Model

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu',input_shape=xx_train[0].shape))
model.add(BatchNormalization())
# model.add(MaxPool2D(2,2))
model.add(Dropout(0.5))
# model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
# # compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               188226048 
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 188,240,470
Trainable params: 188,240,214
Non-trainable params: 256
______________________________________

There is an error due to incompatible dimensions of the dataset and the function due to the lack resources, very limited access to powerful hardawre which can do these computations in a very short amount of time. Also since we split the dataset into multiple subsets of dataset to be able to execute the algorithm in the defined time. 


In [ ]:
history = model.fit(xx_train, yy_train, epochs=10, batch_size=100, validation_data=(xx_test, yy_test), verbose=0)
# evaluate model
_, acc = model.evaluate(xx_test, yy_test, verbose=0)
print('> %.3f' % (acc * 100.0))
# append scores
scores.append(acc)
histories.append(history)

ValueError: ignored

### Plotting the learning Curve

In [ ]:
def plot_learningCurve(history, epoch):
  # Plot training & validation accuracy values
  epoch_range = range(1, epoch+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

plot_learningCurve(history, 5)